In [1]:
import pandas as pd
import pkuseg
import numpy as np
import scipy.sparse as sp

In [2]:
ti_matrix = sp.load_npz('seg_sparse_result.npz')#tf-idf matrix读入

#分词词表读入
temp_word_list = pd.read_csv('seg_word_list.csv',encoding='utf-8')
word_list = temp_word_list.iloc[:,0].tolist()

title_word_list = [w for w in word_list if 'ttt_' in w]
note_word_list = [w for w in word_list if 'nnn_' in w]
title_seg_result = ti_matrix[:,0:len(title_word_list)]
note_seg_result = ti_matrix[:,len(title_word_list):]

#接下来需要分别检索title和note中关键字出现的情况，所以需要删除前缀

new_title_word_list = [w[4:] for w in title_word_list]
new_note_word_list = [w[4:] for w in note_word_list]

#设置新数据结构
vocab_title = pd.Series(range(len(new_title_word_list)))
vocab_title.index = new_title_word_list
vocab_note = pd.Series(range(len(new_note_word_list)))
vocab_note.index = new_note_word_list

#原始文本读入
raw_text = pd.read_csv('./raw_text_data_withseg.csv',encoding='utf-8')
raw_text['URL'] = r'https://travel.qunar.com/travelbook/note/' + raw_text['id'].astype('str')
raw_text = raw_text[['id','URL','标题','note_text']]


In [9]:
def calculate_tfidf_score(w_list,tfidf_matrix,vocab_series):
    '''
    输入：
        w_list：关键字列表
        tfidf_matrix：tfidf得分矩阵
        vocab_series：格式为pd.Series的词表，index为词名称

    输出：
        每篇游记的得分
    '''
    nrows = tfidf_matrix.shape[0]#行数记录
    if len(w_list) == 0:
        print('筛选后列表为空，故tfidf分数都为0')
        return np.zeros((nrows,1))
    else:
        col_index = vocab_series[w_list].tolist()#列索引获取
        tfidf_cols = tfidf_matrix[:,col_index].toarray()
        scores = tfidf_cols.sum(axis=1)
        return scores

def return_most_similar5(keywords,tfidf_title,tfidf_note,raw_df):
    '''
    输入：
        keywords：一段关键字
        tfidf_title：标题的tfidf分数
        tfidf_note：游记文本的tfidf分数
        raw_df：最原始的数据集，包含id，标题，note_text
    输出：
        最相关的5篇游记
    
    '''
    seg = pkuseg.pkuseg(model_name='tourism')
    key_split = seg.cut(keywords)
#     print('分词结果')
#     print(key_split)
    
    #判断关键字是否在所爬数据中出现过
    key_in_vocab_title = []
    key_in_vocab_note = []
    for each_kw in key_split:
        if each_kw in new_title_word_list:
            key_in_vocab_title.append(each_kw)
        else:
            print(each_kw + ' not in vocab_title')
        
        if each_kw in new_note_word_list:
            key_in_vocab_note.append(each_kw)
        else:
            print(each_kw + ' not in vocab_note')
        
    scores_title = calculate_tfidf_score(key_in_vocab_title,tfidf_title,vocab_title)
    scores_note = calculate_tfidf_score(key_in_vocab_note,tfidf_note,vocab_note)
    scores_total = scores_note + scores_title#总得分计算
    raw_df['score'] = scores_total
    raw_df = raw_df.sort_values(by='score',ascending=False)
    raw_df = raw_df.reset_index(drop=True)
    return raw_df.iloc[0:5,:]#返回前5个样本

In [15]:
temp_try = '情侣 双人 浪漫 旅行'
s = return_most_similar5(temp_try,title_seg_result,note_seg_result,raw_text)
s

,id,URL,标题,note_text,score
0,6521801,https://travel.qunar.com/travelbook/note/6521801,一场算不上旅行的旅行,"前言对于海边:一直对落日下的海边漫步充满着无限的幻想:海风吹乱了发丝,空气重暖暖的伴着海洋的...",1.163581
1,6998849,https://travel.qunar.com/travelbook/note/6998849,来一场走说就走的旅行,旅行让人知得失旅行让人看视界旅行让人省自身旅行让你开阔视野，打开格局，遥望未来 ...,1.153971
2,7030479,https://travel.qunar.com/travelbook/note/7030479,三亚，是我欠闺蜜的一次浪漫旅行,和闺蜜相约的旅行，和闺蜜约定一场向往已久的旅行，是生命里最美好的事。生命里总会有这样一个人，...,1.128501
3,6678752,https://travel.qunar.com/travelbook/note/6678752,《跟我去旅行之一路向南》,继《行走八国，做不完一场梦》之后，我从马来西亚回到北京休整了三天又开始上路了，上一部游记，其...,1.104686
4,5727931,https://travel.qunar.com/travelbook/note/5727931,一场说走就走的旅行,对于不经常出门的伙伴，一场远门的旅途总是让人有些许担忧，所以自己出门前也是做足了功课，搜寻了...,1.102160


In [16]:
temp_try = '上海2日迪士尼'
s = return_most_similar5(temp_try,title_seg_result,note_seg_result,raw_text)
s

,id,URL,标题,note_text,score
0,6597241,https://travel.qunar.com/travelbook/note/6597241,Forever21#上海迪士尼Wow旅！,节假日绝对不出游——是我一直以来的宗旨。然而这次选择在一年当中最最黄金的十一出游，实属无奈之...,2.005515
1,7583942,https://travel.qunar.com/travelbook/note/7583942,上海迪士尼4天3晚,"我和宝贝都是迪士尼迷,上海迪士尼建成后,毫不犹豫地带她去了。\n出发前一定要提前做好行程,这...",1.994354
2,6873771,https://travel.qunar.com/travelbook/note/6873771,上海2日游,上海一直是这几年想要去的一个地方，之前北京都去过最少十次，所以想去看看大上海！上海的交通是真...,1.839782
3,6840673,https://travel.qunar.com/travelbook/note/6840673,我的上海迪士尼之旅一日游~~~,2017年6月2日晚上到达上海，第二天开始了我与弟弟、妹妹的迪士尼之旅，好开森的一天~\n1...,1.740377
4,7007799,https://travel.qunar.com/travelbook/note/7007799,【上海迪士尼游记攻略】2天1夜深度探索上海迪士尼乐园,每个人都有童年，每个人童年时都会有一个梦，都会有自己喜欢的童话人物。对于现在的儿童，他们最喜...,1.705452


In [17]:
temp_try = '三亚 浪漫双人'
s = return_most_similar5(temp_try,title_seg_result,note_seg_result,raw_text)
s

,id,URL,标题,note_text,score
0,6585559,https://travel.qunar.com/travelbook/note/6585559,梦一般的浪漫《三亚》,从三亚回来已经两天了，老公和身边的好朋友都催促我把这次三亚之行记录下来，可是我一直磨蹭着。这...,1.587480
1,6779999,https://travel.qunar.com/travelbook/note/6779999,想拥有一个美丽三亚、浪漫的家,生活二十几年的冷空气中，天天与雾霾打交道，却和雾霾成不了一家人，就当海南三亚出现在大家的眼球...,1.566201
2,6636644,https://travel.qunar.com/travelbook/note/6636644,说走就走—三亚浪漫之旅,说说这次旅行 身为喜欢旅行我为什么会选择这么一次旅行、在这里就给大家好好的介绍写的不好的地...,1.550314
3,6993625,https://travel.qunar.com/travelbook/note/6993625,这一次，我来给三亚划个重点,七年去了七次三亚做完了今天的最后一台手术，艰难的推开医院厚重的玻璃门，一股属于腊月里最刺骨的...,1.506633
4,5633861,https://travel.qunar.com/travelbook/note/5633861,三亚游记,三亚是一直一来就是我梦寐以求的一个地方。，有一天晚上闺蜜说，咱们出去玩吧，我问去哪，她说跟团...,1.498767


In [18]:
temp_try = '南京四日自驾游'
s = return_most_similar5(temp_try,title_seg_result,note_seg_result,raw_text)
s

,id,URL,标题,note_text,score
0,5221000,https://travel.qunar.com/travelbook/note/5221000,南京四天三夜自驾游,南京，公元前472年，越王勾践令范蠡筑城于今南京中华门外，这是在南京建造最早的一座城池。公元...,1.723432
1,5488765,https://travel.qunar.com/travelbook/note/5488765,南京四日游,来了西安，才了解中国；而到了南京，才理解中国。自古以来，南京 总是笼罩着一种烟雾之中的伤怀之...,1.650777
2,5111966,https://travel.qunar.com/travelbook/note/5111966,南京，南京,"南京，六朝古都,听起来就是一个有故事的地方，想想就很吸引人呢~趁着现在在安徽工作，离南京不是...",1.626020
3,7469063,https://travel.qunar.com/travelbook/note/7469063,南京四日游,交通蛮方便，三号线夫子庙站下来，走五分钟就到，满街都是玲琅满目的小吃店和纪念品商店，人很多，...,1.570872
4,4821884,https://travel.qunar.com/travelbook/note/4821884,南京旅游,南京,1.538103
